## Fine Tune VGG for your dataset
We start with VGG-16 network pretrained on Imagenet Dataset for 1000 categories. Modify the last fully connected layer according to number of classes in custom dataset.

**Keras with Tensorflow as backend is used here.**

In order to train the model provide train and validation files. These file should be of the form

```train.txt``` <br>
```path/to/image1 label1```<br>
```path/to/image2 label2```<br>
```path/to/image3 label3```

In [2]:
#Starting with some cool imports
from keras.applications.vgg16 import (
    VGG16, preprocess_input, decode_predictions)
from keras.models import Sequential
from keras.preprocessing import image
from keras.layers import Dropout, Flatten, Dense
from keras.models import Model
from keras import optimizers
from keras import utils
from keras.callbacks import ModelCheckpoint,TensorBoard
import numpy as np

Using TensorFlow backend.


### Get the pretrained VGG

In [3]:
def get_vgg_model(num_classes):
    
    #take keras pretrained model and remove last 3 fully connected layers
    vgg = VGG16(weights='imagenet', include_top=False, input_shape=(224, 224, 3))

    #add the custom fc for your num_classes
    last_conv = vgg.output

    x = Flatten()(last_conv)
    x = Dense(2048, activation='relu')(x)
    x = Dropout(0.5)(x)
    x = Dense(num_classes, activation='softmax')(x)

    model = Model(vgg.input, x)

    #only fine tune these fully connected layers
    for layer in model.layers[:19]:
#         print(layer)
        layer.trainable = False

    return model

### Helper function to create train and validation batches

In [4]:
#This class takes list of (image_path label) and returns a batches of specified size.
# get_data function changes the input image size to VGG format(224x224x3) and applies normalization. 
class CustomDataGen():

    def __init__(self, dim_x, dim_y, dim_z, num_class, batch_size):
        self.batch_size = batch_size
        self.dim_x = dim_x
        self.dim_y = dim_y
        self.dim_z = dim_z
        self.num_class = num_class

    def randomize_ind(self,data):
        indexes = np.arange(len(data))
        np.random.shuffle(indexes)
        return indexes

    def get_data(self,list):

        X = np.empty((self.batch_size, self.dim_x, self.dim_y, self.dim_z))
        y = np.empty((self.batch_size,self.num_class))

        for id, data in enumerate(list):
            im_path = data.split(' ')[0]
            label = int(data.split(' ')[1])
            img = image.load_img(im_path, target_size=(self.dim_x, self.dim_y))
            
            x = image.img_to_array(img)
            x = np.expand_dims(x, axis=0)
            x = preprocess_input(x)[0]
            X[id,:,:,:] = x

            y_ = utils.to_categorical(label, self.num_class)
            y[id,...] = y_

        return X, y

    def generate_batch(self, data):

        while 1:
            indexes = self.randomize_ind(data)

            num_batch = int(len(indexes)/self.batch_size)
            for batch_id in range(num_batch):
                temp_list = [data[k] for k in indexes[batch_id*self.batch_size:(batch_id+1)*self.batch_size]]
                X,y = self.get_data(temp_list)
                yield X,y


### Helper function to read txt file to list

In [5]:
def read_img_list_from_file(img_dir,file_path):

    data = []
    with open(file_path) as f:
        for line in f:
            data.append(img_dir + line)

    return data


### Main function

In [ ]:

#Get the train and test data
directory = '../images/'

train_data = read_img_list_from_file(directory,'../splits/train0.txt')
val_data = read_img_list_from_file(directory,'../splits/test0.txt')

#Specify the params
batch_size = 100
num_epoch = 30
num_classes = 25
learning_rate = 1e-4

# Get the modified VGG model
model = get_vgg_model(num_classes)

#Specify the loss and optimizer
model.compile(loss='categorical_crossentropy', optimizer=optimizers.SGD(lr=learning_rate, momentum=0.9),
              metrics=['categorical_accuracy'])

#Data generators
training_generator = CustomDataGen(224, 224, 3, num_classes, batch_size).generate_batch(train_data)
validation_generator = CustomDataGen(224, 224, 3, num_classes, len(val_data)).generate_batch(val_data)

#Save the checkpoints whenever improvement in accuracy is seen
file_path = "weights-improvement-{epoch:02d}-{val_categorical_accuracy:.2f}.hdf5"
checkpoint = ModelCheckpoint(file_path, monitor='val_categorical_accuracy', verbose=1, save_best_only=True, mode='max')

#Tensorboard visualization callback. track the losses while running
tensorboard = TensorBoard(log_dir='./logs', histogram_freq=0, write_graph=True, write_images=False)

callbacks_list = [checkpoint, tensorboard]

#Start training
model.fit_generator(generator = training_generator,
                    steps_per_epoch = len(train_data)//batch_size,
                    epochs= num_epoch,
                    validation_data = validation_generator,
                    validation_steps = 1,
                    callbacks=callbacks_list)

Epoch 1/30


### Inference 

In [8]:
num_classes=25
inference_model = get_vgg_model(num_classes)
inference_model.load_weights('/Users/vidit/Thesis/freiburg_groceries_dataset/fine_tune_vgg/weights_base_version.hdf5')

#preprocess the image
test_image_path = '/Users/vidit/Thesis/freiburg_groceries_dataset/images/COFFEE/COFFEE0006.png'
test_img = image.load_img(test_image_path, target_size=(224, 224))
test_img = image.img_to_array(test_img)
test_img = np.expand_dims(x, axis=0)
test_img = preprocess_input(x)

predictions = inference_model.predict(test_img)
predicted_class = np.argmax(predictions)

print('Predicted class:{}'.format(predicted_class))

ValueError: Dimension 1 in both shapes must be equal, but are 2048 and 256 for 'Assign_52' (op: 'Assign') with input shapes: [25088,2048], [25088,256].